<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Collect-cases-script" data-toc-modified-id="Collect-cases-script-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Collect cases script</a></span></li><li><span><a href="#Check-treatment" data-toc-modified-id="Check-treatment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Check treatment</a></span></li><li><span><a href="#Check-data" data-toc-modified-id="Check-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Check data</a></span></li><li><span><a href="#Test-plotting" data-toc-modified-id="Test-plotting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test plotting</a></span></li></ul></div>

#### Collect cases script

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# import paths
# from scripts import get_raw
# df = get_raw.covid_cases()

#### Check treatment

In [3]:
# from scripts import clean_databases
# dfs = clean_databases.treat_all()
# len(dfs)

In [5]:
import pandas as pd
from paths import *

In [6]:
sorted(TREAT_PATH.glob('*.csv'))

[PosixPath('/Users/fernandascovino/Projects/SimulaCorona/data/treated/treated_covid19_SUS_database.csv'),
 PosixPath('/Users/fernandascovino/Projects/SimulaCorona/data/treated/treated_covid19_cases.csv')]

In [7]:
# df_sus = dfs[0]
df_sus = pd.read_csv(TREAT_PATH / 'treated_covid19_SUS_database.csv')
df_sus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 18 columns):
Unnamed: 0                 5570 non-null int64
id_6char                   5570 non-null int64
id_7char                   5570 non-null int64
municipio_sus              3773 non-null object
municipio                  5570 non-null object
uf                         5570 non-null object
populacao                  5570 non-null int64
clinicos                   4559 non-null float64
intensivistas              251 non-null float64
enfermeiros                5190 non-null float64
anestesiologistas          945 non-null float64
ventiladores_existentes    2337 non-null float64
ventiladores_em_uso        2269 non-null float64
estab_c_vent_sus           2294 non-null float64
quantidade_leitos          3542 non-null float64
leitos_sus                 3506 non-null float64
leitos_nao_sus             1651 non-null float64
municipio_uf               5570 non-null object
dtypes: float64(10), int6

In [10]:
# df_sus = dfs[1]
df_cases = pd.read_csv(TREAT_PATH / 'treated_covid19_cases.csv')
df_cases['municipio'] = df_cases['municipio'].fillna('')
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 14 columns):
Unnamed: 0      356 non-null int64
municipio       356 non-null object
confirmed       356 non-null int64
date            356 non-null object
deaths          11 non-null float64
discarded       320 non-null float64
notes           1 non-null object
notified        319 non-null float64
place_type      356 non-null object
source_url      356 non-null object
uf              356 non-null object
suspect         322 non-null float64
last_update     356 non-null object
municipio_uf    329 non-null object
dtypes: float64(4), int64(2), object(8)
memory usage: 39.0+ KB


#### Check data 

In [11]:
# Check to join infos
df_cases[(df_cases['municipio_uf'].isin(df_sus['municipio_uf']) == False) & (df_cases['municipio'] != '')]

,Unnamed: 0,municipio,confirmed,date,deaths,discarded,notes,notified,place_type,source_url,uf,suspect,last_update,municipio_uf


In [12]:
df_cases['place_type'].value_counts()

city     329
state     27
Name: place_type, dtype: int64

In [13]:
df_cases[df_cases['date'] <= '2020-03-20']['confirmed'].sum()

298

In [14]:
df_cases['confirmed'].sum()

384

In [15]:
cities = df_cases['municipio_uf'].value_counts().index
cities[:10]

Index(['LONDRINA PR', 'FOZ DO IGUACU PR', 'CAMPO LARGO PR',
       'SAO JOSE DOS PINHAIS PR', 'SANTO ANTONIO DO PARAISO PR', 'CURITIBA PR',
       'PONTA GROSSA PR', 'MARINGA PR', 'CASCAVEL PR', 'CIANORTE PR'],
      dtype='object')

#### Test plotting

In [19]:
from utils import display_custom_scales

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'), Loader=yaml.FullLoader)
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'), Loader=yaml.FullLoader)

In [20]:
df_cases.loc[df_cases[df_cases['place_type'] == 'city']['confirmed'].idxmax()]

Unnamed: 0                                                     23
municipio                                                Curitiba
confirmed                                                      31
date                                                   2020-03-21
deaths                                                        NaN
discarded                                                      88
notes                                                         NaN
notified                                                      171
place_type                                                   city
source_url      http://www.saude.pr.gov.br/arquivos/File/CORON...
uf                                                             PR
suspect                                                        52
last_update                            2020-03-21 18:49:48.295500
municipio_uf                                          CURITIBA PR
Name: 23, dtype: object

In [24]:
display_custom_scales()

In [71]:
def add_capacity_traces(df, city):
    
    leitos = df_sus[df_sus['municipio_uf'] == city]['quantidade_leitos'].values[0] * 0.22 # assuming 78% in use
    df['leitos_disponiveis'] = leitos

    vent_disponivel = df_sus[df_sus['municipio_uf'] == city]['ventiladores_existentes'].values[0] - df_sus[df_sus['municipio_uf'] == city]['ventiladores_em_uso'].values[0]
    df['ventiladores_disponiveis'] = vent_disponivel
    
    return df

def plot_scenario(city, vent=True, leitos=False):
    
    title = 'Casos confirmados, suspeitos e mortes em {}'.format(city)

    t = df_cases[df_cases['municipio_uf'] == city][['date', 'deaths', 'confirmed']].sort_values('date').groupby('date').sum()
    t = add_capacity_traces(t, city)

    cols = ['deaths', 'confirmed']
    if vent:
        cols.append('ventiladores_disponiveis')
    if leitos:
        cols.append('leitos_disponiveis')
        
    fig = t[cols].iplot(theme='custom', colorscale='impulso', title=iplottitle(title), asFigure=True)
    
    return fig

In [72]:
city = 'CURITIBA PR'
plot_scenario(city, vent=False, leitos=True)